In [3]:
#librerias a usar
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from multiprocessing import Pool, cpu_count


In [4]:
#leer el archivo json

chunks = pd.read_json('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json', lines=True, chunksize=1000)

df_review_crudo = pd.concat(chunks)

In [3]:
#Analizar el 'peso' del archivo

import os

size_in_bytes = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json')
size_in_megabytes = size_in_bytes / 1e6

print(f'La capacidad del archivo "review.json" es de {size_in_megabytes} MB')

La capacidad del archivo "review.json" es de 5341.868833 MB


In [4]:
df_review_crudo.shape

(6990280, 9)

In [5]:
df_review_crudo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [6]:
#verificar si hay valores nulos
df_review_crudo.isnull().sum()


review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [7]:
#verificar si hay valores duplicados
df_review_crudo.duplicated().sum()



### Analisis de las columnas: 

Segun el estudio realizado, nos encontramos con tres columnas en particular que se escapan a nuestro objetivo, las mismas son: 

<u>useful (int64):</u> Es el número de veces que otros usuarios han marcado la reseña como "útil". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u>funny (int64):</u>  Es el número de veces que otros usuarios han marcado la reseña como "divertida". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

<u> cool (int64): </u> Es el número de veces que otros usuarios han marcado la reseña como "genial" o "interesante". Este valor es un número entero.<span style='color:lightpink'>consideramos que no es necesaria a nuestros intereses</span><span style='color:red'> SE PROCEDE A BORRAR LA COLUMNA</span>

In [5]:
#hacemos una copia del dataframe original
df_review = df_review_crudo.copy()

In [6]:
#verificamos la copia
df_review.shape

(6990280, 9)

In [6]:
#y dropeamos las columnas que de 'useful', 'funny' y 'cool' que no vamos a usar
df_review.drop(columns=['useful', 'funny', 'cool'], inplace=True)

In [8]:
#verificamos que se hayan dropeado
df_review.shape

(6990280, 6)

In [7]:
#Funcion para analizar sentimientos utilizando la libreria 'multiprocessing' la cual, en las lineas 29 y 32 prepara la maquina para utilizar todos los nucleos de procesamiento disponibles, menos uno, para realizar la tarea de forma paralela y lo mas rapido posible.

import multiprocess as mp

nltk.download('vader_lexicon')

def analisis_sentimientos(texto):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(texto)

# Function to parallelize the sentiment analysis
def parallelize_dataframe(df, func, num_cores):
    df_split = np.array_split(df, num_cores)
    pool = mp.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def apply_sentiment_analysis(df):
    df['sentimientos'] = df['text'].apply(analisis_sentimientos)
    return df

if __name__ == '__main__':
    # Load your DataFrame
    # df_review = pd.read_csv('path_to_your_file.csv') # Uncomment this if loading from a file
    
    # Obtener el número total de núcleos de procesamiento lógicos en la máquina
    total_cores = mp.cpu_count()

    # Usar todos menos uno para dejar recursos libres para otras tareas
    num_cores_to_use = total_cores - 1

    # Paralelizar el DataFrame
    df_review = parallelize_dataframe(df_review, apply_sentiment_analysis, num_cores=num_cores_to_use)

    # Guardar el resultado si es necesario
    # df_review.to_csv('path_to_save_result.csv', index=False) # Uncomment to save the result

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/luisalbertocerelli/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [9]:
#hacemos un csv de las primeras 1000 filas para ver como quedo
df_review.head(1000).to_csv('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp.df_review_final_1000_filas.csv', index=False)



In [10]:
#guardamos el df en un nuevo archivo parquet
df_review.to_parquet('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet', index=False)



In [11]:
#Analizamos el 'peso' del nuevo archivo parquet
import os

size_in_bytes = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet')
size_in_megabytes = size_in_bytes / 1e6

print(f'La capacidad del archivo parquet "review_listo_para_usar" es de {size_in_megabytes} MB')

La capacidad del archivo parquet "review_listo_para_usar" es de 2999.105605 MB


In [12]:
import os

# Obtén el tamaño del archivo JSON
size_in_bytes_json = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review.json')
size_in_megabytes_json = size_in_bytes_json / 1e6

# Obtén el tamaño del archivo Parquet
size_in_bytes_parquet = os.path.getsize('/Users/luisalbertocerelli/Desktop/00-Todo/Data_Science/01-Full_Time/TPFINAL/Yelp/notebooks_yelp/review_json_yelp/review_listo_para_usar.parquet')
size_in_megabytes_parquet = size_in_bytes_parquet / 1e6

# Calcula la diferencia
diferencia = size_in_megabytes_json - size_in_megabytes_parquet 

print(f'La diferencia de peso entre el archivo json y el parquet es de {diferencia} MB')



La diferencia de peso entre el archivo json y el parquet es de 2342.7632280000003 MB
